In [1]:
from rdkit import Chem
import pandas as pd
import sys
sys.path.append('..')
from utils import *
import glob
import os

In [8]:
metanetx_df = pd.read_csv('metanetx/10May2023_metanetx_chem_prop.tsv', sep='\t', comment='#').dropna(subset=['SMILES'])
metanetx_df
mnx_to_smiles = pd.Series(metanetx_df['SMILES'].values, index=metanetx_df['ID'].values).to_dict()

In [9]:
# organism_name = 'p_aeruginosa'
# organism_name = 'e_coli'
# organism_name = 'p_putida'
# organism_name = 'p_solanacearum'
# organism_name = 'k_pneumoniae'
# organism_name = 'z_mays'
organism_name = 'a_nidulans'
CPDS_FILES = glob.glob(os.path.join(organism_name,"*chemicals.tsv"))
SAVE_NAME = "{}/{}_metabolites_from_metanetx_pathways.csv".format(organism_name, organism_name)

print (CPDS_FILES)

['a_nidulans/rhea_EMENI.chemicals.tsv']


In [10]:
cpds_df = pd.concat([pd.read_csv(file, sep='\t', comment = '#', header=None) for file in CPDS_FILES])
cpds_df

,0,1,2,3,4,5,6
0,MNXM1,H(+),chebi:15378,H,1.00794,1.0,mnx:PROTON;CHEBI:15378;CHEBI:24636;CHEBI:29233...
1,MNXM10,NADH,chebi:57945,C21H27N7O14P2,663.11022,-2.0,chebi:57945;CHEBI:16908;CHEBI:57945;bigg.metab...
2,MNXM100,(2E)-geranyl diphosphate,chebi:58057,C10H17O7P2,311.04660,-3.0,chebi:58057;CHEBI:17211;CHEBI:58057;SLM:000000...
3,MNXM1020,"(4R,5S)-dethiobiotin",chebi:149473,C10H17N2O3,213.12447,-1.0,chebi:149473;CHEBI:149473;CHEBI:16691;CHEBI:42...
4,MNXM103049,berkeleyone A,chebi:69024,C26H38O6,446.26684,0.0,chebi:69024;CHEBI:69024
...,...,...,...,...,...,...,...
398,MNXM9227,versicolorin B,chebi:77951,C18H11O7,339.05103,-1.0,chebi:77951;CHEBI:72674;CHEBI:77951;chebi:7267...
399,MNXM959,2-dehydropantoate,chebi:11561,C6H9O4,145.05063,-1.0,chebi:11561;CHEBI:11561;CHEBI:17094;bigg.metab...
400,MNXM96330,a 2'-deoxyribonucleoside 5'-triphosphate,chebi:61560,C5H8O12P3*,NaN,NaN,chebi:61560;CHEBI:16381;CHEBI:61560;chebi:1412...
401,MNXM984,"(2R,3S)-homoisocitrate",chebi:15404,C7H7O7,203.02082,-3.0,chebi:15404;CHEBI:15404;CHEBI:29094;CHEBI:3090...


In [11]:
cpds_df.columns = ['ID','name','source','formula','mass','charge','xrefs']
cpds_df['smiles'] = [standardize_smiles(mnx_to_smiles[i]) if i in mnx_to_smiles.keys() else None for i in cpds_df['ID']]
cpds_df = cpds_df.dropna(subset='smiles')
cpds_df = cpds_df[cpds_df['smiles'].map(lambda x : '*' not in x)]
cpds_df = cpds_df.drop_duplicates(subset=['smiles'])

cpds_df

[14:29:02] WARNING: not removing hydrogen atom without neighbors
[14:29:02] WARNING: not removing hydrogen atom without neighbors


,ID,name,source,formula,mass,charge,xrefs,smiles
0,MNXM1,H(+),chebi:15378,H,1.00794,1.0,mnx:PROTON;CHEBI:15378;CHEBI:24636;CHEBI:29233...,[H+]
1,MNXM10,NADH,chebi:57945,C21H27N7O14P2,663.11022,-2.0,chebi:57945;CHEBI:16908;CHEBI:57945;bigg.metab...,NC(=O)C1=CN([C@@H]2O[C@H](COP(=O)(O)OP(=O)(O)O...
2,MNXM100,(2E)-geranyl diphosphate,chebi:58057,C10H17O7P2,311.04660,-3.0,chebi:58057;CHEBI:17211;CHEBI:58057;SLM:000000...,CC(C)=CCC/C(C)=C/COP(=O)(O)OP(=O)(O)O
3,MNXM1020,"(4R,5S)-dethiobiotin",chebi:149473,C10H17N2O3,213.12447,-1.0,chebi:149473;CHEBI:149473;CHEBI:16691;CHEBI:42...,C[C@@H]1NC(=O)N[C@@H]1CCCCCC(=O)O
4,MNXM103049,berkeleyone A,chebi:69024,C26H38O6,446.26684,0.0,chebi:69024;CHEBI:69024,C=C1[C@@]2(C)C[C@H]3[C@]4(C)CC[C@@H](O)C(C)(C)...
...,...,...,...,...,...,...,...,...
397,MNXM920,cis-homoaconitate,chebi:58174,C7H5O6,185.01026,-3.0,chebi:58174;CHEBI:17516;CHEBI:58174;bigg.metab...,O=C(O)/C=C(/CCC(=O)O)C(=O)O
398,MNXM9227,versicolorin B,chebi:77951,C18H11O7,339.05103,-1.0,chebi:77951;CHEBI:72674;CHEBI:77951;chebi:7267...,O=C1c2cc(O)cc(O)c2C(=O)c2c1cc1c(c2O)[C@@H]2CCO...
399,MNXM959,2-dehydropantoate,chebi:11561,C6H9O4,145.05063,-1.0,chebi:11561;CHEBI:11561;CHEBI:17094;bigg.metab...,CC(C)(CO)C(=O)C(=O)O
401,MNXM984,"(2R,3S)-homoisocitrate",chebi:15404,C7H7O7,203.02082,-3.0,chebi:15404;CHEBI:15404;CHEBI:29094;CHEBI:3090...,O=C(O)CC[C@H](C(=O)O)[C@@H](O)C(=O)O


In [12]:
## Uncommment to save

pd.DataFrame({'name':cpds_df['name'].tolist(), 'smiles':cpds_df['smiles'].tolist()}).to_csv(SAVE_NAME, index=False, sep='\t')
print ("DATA SAVED TO", SAVE_NAME)

DATA SAVED TO a_nidulans/a_nidulans_metabolites_from_metanetx_pathways.csv


In [16]:
cpds = pd.DataFrame({'name':cpds_df['name'].tolist(), 'smiles':cpds_df['smiles'].tolist()})
cpds[cpds['name'].map(lambda x : 'emodin' in x.lower())]

,name,smiles
92,emodin,Cc1cc(O)c2c(c1)C(=O)c1cc(O)cc(O)c1C2=O
93,emodin anthrone,Cc1cc(O)c2c(c1)Cc1cc(O)cc(O)c1C2=O


In [8]:
metanetx_df['std_smiles'] = [standardize_smiles(s) for s in metanetx_df['SMILES']]

[10:59:46] WARNING: not removing hydrogen atom without neighbors
[10:59:46] WARNING: not removing hydrogen atom without neighbors
[10:59:46] WARNING: not removing hydrogen atom without neighbors
[10:59:46] WARNING: not removing hydrogen atom without neighbors
[11:00:00] WARNING: not removing hydrogen atom without neighbors
[11:00:00] WARNING: not removing hydrogen atom without neighbors
[11:00:00] WARNING: not removing hydrogen atom without neighbors
[11:00:00] WARNING: not removing hydrogen atom without neighbors
[11:00:00] WARNING: not removing hydrogen atom without neighbors
[11:00:00] WARNING: not removing hydrogen atom without neighbors
[11:00:00] WARNING: not removing hydrogen atom without neighbors
[11:00:00] WARNING: not removing hydrogen atom without neighbors
[11:00:01] WARNING: not removing hydrogen atom without neighbors
[11:00:01] WARNING: not removing hydrogen atom without neighbors
[11:00:03] SMILES Parse Error: extra open parentheses for input: 'COc1cc(C'
[11:00:03] SMI

[11:00:47] Explicit valence for atom # 33 Na, 3, is greater than permitted
[11:00:47] WARNING: not removing hydrogen atom without neighbors
[11:00:47] WARNING: not removing hydrogen atom without neighbors
[11:00:47] SMILES Parse Error: extra open parentheses for input: 'C[C@@](O)(CS(=O)(=O)c1ccc(F)cc1)C(=O)Nc1ccc(C'
[11:00:47] SMILES Parse Error: extra open parentheses for input: 'C[C@@](O)(CS(=O)(=O)c1ccc(F)cc1)C(=O)Nc1ccc(C'
[11:00:47] SMILES Parse Error: extra open parentheses for input: 'C[C@@](O)(CS(=O)(=O)c1ccc(F)cc1)C(=O)Nc1ccc(C'
[11:00:47] SMILES Parse Error: extra open parentheses for input: 'C[C@](O)(CS(=O)(=O)c1ccc(F)cc1)C(=O)Nc1ccc(C'
[11:00:47] SMILES Parse Error: extra open parentheses for input: 'C[C@](O)(CS(=O)(=O)c1ccc(F)cc1)C(=O)Nc1ccc(C'
[11:00:47] SMILES Parse Error: extra open parentheses for input: 'C[C@](O)(CS(=O)(=O)c1ccc(F)cc1)C(=O)Nc1ccc(C'
[11:00:49] SMILES Parse Error: extra open parentheses for input: 'C[N+](C)([O-])CCCC1(c2ccc(F)cc2)OCc2cc(C'
[11:00:49] S

[11:00:58] SMILES Parse Error: extra open parentheses for input: 'C[C@H]1CCN(C(=O)CC'
[11:00:58] SMILES Parse Error: extra open parentheses for input: 'C[C@H]1CCN(C(=O)CC'
[11:00:58] SMILES Parse Error: extra open parentheses for input: 'C[C@H]1CCN(C(=O)CC'
[11:00:58] SMILES Parse Error: extra open parentheses for input: 'COc1ccc(CN2C3CC2CN(c2ccc(-c4cc(OCC(C)(C)O)cn5ncc(C'
[11:00:58] SMILES Parse Error: extra open parentheses for input: 'COc1ccc(CN2C3CC2CN(c2ccc(-c4cc(OCC(C)(C)O)cn5ncc(C'
[11:00:58] SMILES Parse Error: extra open parentheses for input: 'COc1ccc(CN2C3CC2CN(c2ccc(-c4cc(OCC(C)(C)O)cn5ncc(C'
[11:00:58] SMILES Parse Error: extra open parentheses for input: 'CN1CCC(COc2cnc(-c3cccc(Cn4nc(-c5cccc(C'
[11:00:58] SMILES Parse Error: extra open parentheses for input: 'CN1CCC(COc2cnc(-c3cccc(Cn4nc(-c5cccc(C'
[11:00:58] SMILES Parse Error: extra open parentheses for input: 'CN1CCC(COc2cnc(-c3cccc(Cn4nc(-c5cccc(C'
[11:00:58] SMILES Parse Error: extra open parentheses for input: 'C=CC

[11:01:01] SMILES Parse Error: extra open parentheses for input: 'CC1(C)C(/C=C(\Cl)C(F)(F)F)C1(O)C(=O)OC(C'
[11:01:01] SMILES Parse Error: extra open parentheses for input: 'CC1(C)C(/C=C(\Cl)C(F)(F)F)C1(O)C(=O)OC(C'
[11:01:01] SMILES Parse Error: extra open parentheses for input: 'CC1(C)C(/C=C(\Cl)C(F)(F)F)C1(O)C(=O)OC(C'
[11:01:01] SMILES Parse Error: extra open parentheses for input: 'C=CC(O)(C(=O)C(=O)[O-])C(C'
[11:01:01] SMILES Parse Error: extra open parentheses for input: 'C=CC(O)(C(=O)C(=O)[O-])C(C'
[11:01:01] SMILES Parse Error: extra open parentheses for input: 'C=CC(O)(C(=O)C(=O)[O-])C(C'
[11:01:01] SMILES Parse Error: extra open parentheses for input: 'CNC(=O)c1cc(C'
[11:01:01] SMILES Parse Error: extra open parentheses for input: 'CNC(=O)c1cc(C'
[11:01:01] SMILES Parse Error: extra open parentheses for input: 'CNC(=O)c1cc(C'
[11:01:01] Explicit valence for atom # 25 B, 5, is greater than permitted
[11:01:01] SMILES Parse Error: extra open parentheses for input: 'CC(CO)(CC(=

[11:01:05] Explicit valence for atom # 25 B, 5, is greater than permitted
[11:01:05] SMILES Parse Error: extra open parentheses for input: 'Oc1ccc(C'
[11:01:05] SMILES Parse Error: extra open parentheses for input: 'Oc1ccc(C'
[11:01:05] SMILES Parse Error: extra open parentheses for input: 'Oc1ccc(C'
[11:01:05] SMILES Parse Error: extra open parentheses for input: 'CS(=O)(=NC'
[11:01:05] SMILES Parse Error: extra open parentheses for input: 'CS(=O)(=NC'
[11:01:05] SMILES Parse Error: extra open parentheses for input: 'CS(=O)(=NC'
[11:01:05] SMILES Parse Error: extra open parentheses for input: 'CC1(CO)C(C=CCl)C1C(=O)OC(C'
[11:01:05] SMILES Parse Error: extra open parentheses for input: 'CC1(CO)C(C=CCl)C1C(=O)OC(C'
[11:01:05] SMILES Parse Error: extra open parentheses for input: 'CC1(CO)C(C=CCl)C1C(=O)OC(C'
[11:01:05] SMILES Parse Error: extra open parentheses for input: 'Cc1cc(C'
[11:01:05] SMILES Parse Error: extra open parentheses for input: 'Cc1cc(C'
[11:01:05] SMILES Parse Error: e

...

[11:05:10] WARNING: not removing hydrogen atom without neighbors
[11:05:10] WARNING: not removing hydrogen atom without neighbors
[11:05:11] SMILES Parse Error: extra open parentheses for input: 'CC(O)(CS(=O)(=O)c1ccc(F)cc1)C(=O)Nc1ccc(C'
[11:05:11] SMILES Parse Error: extra open parentheses for input: 'CC(O)(CS(=O)(=O)c1ccc(F)cc1)C(=O)Nc1ccc(C'
[11:05:11] SMILES Parse Error: extra open parentheses for input: 'CC(O)(CS(=O)(=O)c1ccc(F)cc1)C(=O)Nc1ccc(C'
[11:05:11] SMILES Parse Error: extra open parentheses for input: 'C[C@@H]1C[C@H](C)[C@@H](O)CC(=O)O[C@@H]([C@@H]2CCC[C@H]2C(=O)O)C/C=C/C=C(/C'
[11:05:11] SMILES Parse Error: extra open parentheses for input: 'C[C@@H]1C[C@H](C)[C@@H](O)CC(=O)O[C@@H]([C@@H]2CCC[C@H]2C(=O)O)C/C=C/C=C(/C'
[11:05:11] SMILES Parse Error: extra open parentheses for input: 'C[C@@H]1C[C@H](C)[C@@H](O)CC(=O)O[C@@H]([C@@H]2CCC[C@H]2C(=O)O)C/C=C/C=C(/C'
[11:05:11] SMILES Parse Error: extra open parentheses for input: 'C[C@@H]1C[C@H](C)[C@@H](O)CC(=O)OC([C@@H]2CCC[C@

[11:05:14] SMILES Parse Error: extra open parentheses for input: 'CS(=O)(=O)c1cc(C(F)(F)F)ccc1C(=O)C(C'
[11:05:14] SMILES Parse Error: extra open parentheses for input: 'CS(=O)(=O)c1cc(C(F)(F)F)ccc1C(=O)C(C'
[11:05:14] SMILES Parse Error: extra open parentheses for input: 'CS(=O)(=O)c1cc(C(F)(F)F)ccc1C(=O)C(C'
[11:05:14] SMILES Parse Error: extra open parentheses for input: 'Cc1nn(C)c(C(O)=C(C'
[11:05:14] SMILES Parse Error: extra open parentheses for input: 'Cc1nn(C)c(C(O)=C(C'
[11:05:14] SMILES Parse Error: extra open parentheses for input: 'Cc1nn(C)c(C(O)=C(C'
[11:05:14] SMILES Parse Error: extra open parentheses for input: 'CC1=CCC(CC'
[11:05:14] SMILES Parse Error: extra open parentheses for input: 'CC1=CCC(CC'
[11:05:14] SMILES Parse Error: extra open parentheses for input: 'CC1=CCC(CC'
[11:05:14] SMILES Parse Error: extra open parentheses for input: 'CC(=O)Nc1cc(Nc2cc(NC3CC3)n3ncc(C'
[11:05:14] SMILES Parse Error: extra open parentheses for input: 'CC(=O)Nc1cc(Nc2cc(NC3CC3)n3ncc

[11:05:46] SMILES Parse Error: extra open parentheses for input: 'CCOc1cc2ncc(C'
[11:05:46] SMILES Parse Error: extra open parentheses for input: 'CCOc1cc2ncc(C'
[11:05:46] SMILES Parse Error: extra open parentheses for input: 'CCOc1cc2ncc(C'
[11:05:46] SMILES Parse Error: extra open parentheses for input: 'COC(=O)c1cccc(CCN(C)CCC[C@@](C'
[11:05:46] SMILES Parse Error: extra open parentheses for input: 'COC(=O)c1cccc(CCN(C)CCC[C@@](C'
[11:05:46] SMILES Parse Error: extra open parentheses for input: 'COC(=O)c1cccc(CCN(C)CCC[C@@](C'
[11:05:46] WARNING: not removing hydrogen atom without neighbors
[11:05:46] WARNING: not removing hydrogen atom without neighbors
[11:05:46] WARNING: not removing hydrogen atom without neighbors
[11:05:46] WARNING: not removing hydrogen atom without neighbors
[11:05:46] WARNING: not removing hydrogen atom without neighbors
[11:05:46] WARNING: not removing hydrogen atom without neighbors
[11:05:46] WARNING: not removing hydrogen atom without neighbors
[11:05:4

[11:05:47] SMILES Parse Error: extra open parentheses for input: 'CC1=C2[N+]3=C(C=C4[N+]5=C(C(C)=C6N7[C@@H]8[C@H](CC(N)=O)[C@]6(C)CCC(=O)NC[C@H](C)OP(=O)([O-])O[C@@H]6[C@@H](CO)O[C@@H]([C@@H]6O)n6c[n+](c9ncnc(N)c96)[Co-3]753(C'
[11:05:47] SMILES Parse Error: extra open parentheses for input: 'CC1=C2[N+]3=C(C=C4[N+]5=C(C(C)=C6N7[C@@H]8[C@H](CC(N)=O)[C@]6(C)CCC(=O)NC[C@H](C)OP(=O)([O-])O[C@@H]6[C@@H](CO)O[C@@H]([C@@H]6O)n6c[n+](c9ncnc(N)c96)[Co-3]753(C'
[11:05:47] SMILES Parse Error: extra open parentheses for input: 'CC1=C2[N+]3=C(C=C4[N+]5=C(C(C)=C6N7[C@@H]8[C@H](CC(N)=O)[C@]6(C)CCC(=O)NC[C@H](C)OP(=O)([O-])O[C@@H]6[C@@H](CO)O[C@@H]([C@@H]6O)n6c[n+](c9ncnc(N)c96)[Co-3]753(C'
[11:05:47] SMILES Parse Error: extra open parentheses for input: 'CC1=C2[N+]3=C(C=C4[N+]5=C(C(C)=C6N7[C@@H]8[C@H](CC(N)=O)[C@]6(C)CCC(=O)NC[C@H](C)OP(=O)([O-])O[C@@H]6[C@@H](CO)O[C@@H]([C@@H]6O)n6c[n+](c9nc(C)nc(N)c96)[Co-3]753(C'
[11:05:47] SMILES Parse Error: extra open parentheses for input: 'CC1=C2[N+]3=C(C=C4[

[11:05:47] SMILES Parse Error: extra open parentheses for input: 'CC1(C)CCS(=O)c2ccc(C'
[11:05:47] SMILES Parse Error: extra open parentheses for input: 'CC1(C)CCS(=O)c2ccc(C'
[11:05:47] SMILES Parse Error: extra open parentheses for input: 'CC1(C)CCS(=O)c2ccc(C'
[11:05:49] WARNING: not removing hydrogen atom without neighbors
[11:05:49] WARNING: not removing hydrogen atom without neighbors
[11:05:56] SMILES Parse Error: extra open parentheses for input: 'Cn1c(C'
[11:05:56] SMILES Parse Error: extra open parentheses for input: 'Cn1c(C'
[11:05:56] SMILES Parse Error: extra open parentheses for input: 'Cn1c(C'
[11:05:57] SMILES Parse Error: extra open parentheses for input: 'C[C@H](NC(=O)C(C)(C)Oc1ccc(C(F)(F)F)cn1)[C@@H](Cc1ccc(Cl)cc1)c1cccc(C'
[11:05:57] SMILES Parse Error: extra open parentheses for input: 'C[C@H](NC(=O)C(C)(C)Oc1ccc(C(F)(F)F)cn1)[C@@H](Cc1ccc(Cl)cc1)c1cccc(C'
[11:05:57] SMILES Parse Error: extra open parentheses for input: 'C[C@H](NC(=O)C(C)(C)Oc1ccc(C(F)(F)F)cn1)[C@@

In [9]:
deduped = metanetx_df.drop_duplicates(subset='std_smiles')

In [10]:
deduped['mol'] = [Chem.MolFromSmiles(s) for s in deduped['std_smiles']]

[11:07:36] WARNING: not removing hydrogen atom without neighbors
[11:07:40] WARNING: not removing hydrogen atom without neighbors
[11:07:40] WARNING: not removing hydrogen atom without neighbors
[11:07:40] WARNING: not removing hydrogen atom without neighbors
[11:07:40] WARNING: not removing hydrogen atom without neighbors
[11:07:41] WARNING: not removing hydrogen atom without neighbors
[11:07:41] SMILES Parse Error: extra open parentheses for input: 'COc1cc(C'
[11:07:41] SMILES Parse Error: extra open parentheses for input: 'CC(=O)OCC1=C(C(=O)O)N2C(=O)[C@@H](NC(=O)CC'
[11:07:43] WARNING: not removing hydrogen atom without neighbors
[11:07:43] WARNING: not removing hydrogen atom without neighbors
[11:07:43] SMILES Parse Error: extra open parentheses for input: 'C[C@](O)(COc1ccc(C'
[11:07:43] SMILES Parse Error: extra open parentheses for input: 'CNC(=O)c1ccc(N2C(=S)N(c3ccc(C'
[11:07:44] WARNING: not removing hydrogen atom without neighbors
[11:07:44] WARNING: not removing hydrogen atom

[11:08:01] SMILES Parse Error: unclosed ring for input: 'COC1=C(C(=O)[O-])[C@H](C)[C@@]23O[C@]24c2cc(O)c(C(=O)c5c(O)ccc(O)c5C(=O)[O-])c(O)c2N[C@H]3C'
[11:08:01] SMILES Parse Error: extra open parentheses for input: 'CC(C)C(C(=O)OC(C'
[11:08:01] WARNING: not removing hydrogen atom without neighbors
[11:08:01] SMILES Parse Error: extra open parentheses for input: 'CN(C)CC(O)C(O)C1(c2ccc(F)cc2)OCc2cc(C'
[11:08:01] SMILES Parse Error: extra open parentheses for input: 'CC(O)(CO)C(C(=O)OC(C'
[11:08:01] SMILES Parse Error: extra open parentheses for input: 'CC(=O)C(C'
[11:08:01] SMILES Parse Error: extra open parentheses for input: 'COCC(C(=O)[O-])c1ccccc1Oc1cc(Oc2ccc(O)c(O)c2C'
[11:08:01] SMILES Parse Error: extra open parentheses for input: 'CC(C=O)C(C(=O)OC(C'
[11:08:01] SMILES Parse Error: extra open parentheses for input: 'CC1(C)C(C(=O)OC(C'
[11:08:01] SMILES Parse Error: extra open parentheses for input: 'CO/C=C(/C(=O)OC)c1ccccc1Oc1cc(Oc2c(O)c(O)cc(O)c2C'
[11:08:01] SMILES Parse Error:

[11:08:05] SMILES Parse Error: extra open parentheses for input: 'COC=C(C(=O)[O-])c1ccccc1Oc1cc(O/C(=C\C'
[11:08:05] SMILES Parse Error: extra open parentheses for input: 'CC(C)(CO)C(O)(C'
[11:08:05] SMILES Parse Error: extra open parentheses for input: 'CC1(C)C(C=C(Cl)C(F)(F)F)C1(O)C(=O)OC(C'
[11:08:05] SMILES Parse Error: extra open parentheses for input: 'CC1(C)C(CC(Cl)Cl)C1C(=O)OC(C'
[11:08:05] SMILES Parse Error: extra open parentheses for input: 'COCC(C(=O)[O-])c1c(Oc2cc(Oc3ccccc3C'
[11:08:05] SMILES Parse Error: unclosed ring for input: 'COC1=C(C(=O)[O-])[C@H](C)[C@@]23O[C@]24c2cc(O)c5c(=O)c6c(O)cc(O)c(O)c6oc(=O)c5c2N[C@H]3C'
[11:08:05] SMILES Parse Error: extra open parentheses for input: 'Cn1c(-c2cc(Br)nn2-c2nc(O)c(O)cc2O)nc2c(CO)cc(C'
[11:08:05] SMILES Parse Error: extra open parentheses for input: 'CCCCC(CC)COC(=O)C(C'
[11:08:05] WARNING: not removing hydrogen atom without neighbors
[11:08:05] WARNING: not removing hydrogen atom without neighbors
[11:08:05] SMILES Parse Erro

[11:08:10] SMILES Parse Error: extra open parentheses for input: 'CC1(CO)C(CC(Cl)Cl)C1C(=O)OC(C'
[11:08:11] SMILES Parse Error: extra open parentheses for input: 'CCOCn1c(-c2cc(O)c(O)c(O)c2)c(C'
[11:08:11] SMILES Parse Error: extra open parentheses for input: 'CCOCn1c(-c2cc(O)c(O)cc2O)c(C'
[11:08:11] SMILES Parse Error: extra open parentheses for input: 'COC1=C(C(=O)[O-])[C@H](CO)[C@@]23O[C@]24c2cc(O)c5c(c2N[C@H]3C'
[11:08:11] SMILES Parse Error: extra open parentheses for input: 'CO/C=C(/C(=O)[O-])c1c(Oc2cc(Oc3ccccc3C'
[11:08:11] SMILES Parse Error: extra open parentheses for input: 'CO/C=C(/C(=O)[O-])c1ccccc1Oc1cc(Oc2c(C'
[11:08:11] SMILES Parse Error: extra open parentheses for input: 'CC1(C)C(C=CBr)C1C(=O)OC(C'
[11:08:11] SMILES Parse Error: extra open parentheses for input: 'CN(C)CC(O)CC1(c2ccc(F)cc2)OC(O)c2cc(C'
[11:08:11] SMILES Parse Error: extra open parentheses for input: 'CC(=O)Nc1ccc(NCC(O)C'
[11:08:11] SMILES Parse Error: extra open parentheses for input: 'CO/C=C\c1c(Oc2cc

[11:08:17] SMILES Parse Error: extra open parentheses for input: 'CC1(CO)C(C=CC(F)(F)F)C1C(=O)OC(C'
[11:08:17] SMILES Parse Error: unclosed ring for input: 'COC1=C(C(=O)[O-])[C@H](C)[C@@]23O[C@]24c2cc(O)c5oc(=O)c6c(O)ccc(O)c6c(=O)c5c2N[C@H]3C'
[11:08:17] SMILES Parse Error: unclosed ring for input: 'COC1=C(C(=O)[O-])[C@H](C)[C@@]23O[C@@]2(c2cc(O)c4c(c2N)C(=O)c2c(O)c(O)cc(O)c2C4=O)[C@@H]1C'
[11:08:17] SMILES Parse Error: extra open parentheses for input: 'CC1(C)C(O)(/C=C/C(F)(F)F)C1(O)C(=O)OC(C'
[11:08:17] SMILES Parse Error: extra open parentheses for input: 'CCCCCCC(=O)C(=O)Oc1c(Br)cc(C'
[11:08:17] SMILES Parse Error: extra open parentheses for input: 'CC1(C=O)C(CC(Cl)C(F)(F)F)C1C(=O)OC(C'
[11:08:17] SMILES Parse Error: unclosed ring for input: 'COC1=C(C(=O)[O-])[C@H](C)[C@@]23O[C@]24c2cc(O)c5c(=O)c6c(O)ccc(O)c6oc(=O)c5c2N[C@H]3C'
[11:08:17] SMILES Parse Error: extra open parentheses for input: 'COc1ccccc1N1CCSC1=C(C'
[11:08:17] WARNING: not removing hydrogen atom without neighbors
[1

...

[11:09:15] SMILES Parse Error: extra open parentheses for input: 'COc1cccc(S(=O)(=O)N2CCCCN3[C@@H](CO)[C@@H](c4ccc(C'
[11:09:15] SMILES Parse Error: extra open parentheses for input: 'O=S(=O)(c1ccccc1)N1CCCCN2[C@@H](CO)[C@@H](c3ccc(C'
[11:09:15] SMILES Parse Error: extra open parentheses for input: 'Cc1ccccc1S(=O)(=O)N1CCCCN2[C@@H](CO)[C@@H](c3ccc(C'
[11:09:15] SMILES Parse Error: extra open parentheses for input: 'O=S(=O)(c1ccccc1)N1CCCCN2[C@H](CO)[C@@H](c3ccc(C'
[11:09:15] SMILES Parse Error: extra open parentheses for input: 'CCS(=O)(=O)N1CCCCN2[C@H](CO)[C@@H](c3ccc(C'
[11:09:15] SMILES Parse Error: extra open parentheses for input: 'Cc1ccccc1S(=O)(=O)N1CCCCN2[C@H](CO)[C@@H](c3ccc(C'
[11:09:15] SMILES Parse Error: extra open parentheses for input: 'COc1cccc(S(=O)(=O)N2CCCCN3[C@H](C2)[C@@H](c2ccc(C'
[11:09:15] SMILES Parse Error: extra open parentheses for input: 'O=S(=O)(c1ccccc1)N1CCCCN2[C@H](C1)[C@@H](c1ccc(C'
[11:09:15] SMILES Parse Error: extra open parentheses for input: 'CCS(=

[11:09:16] SMILES Parse Error: extra open parentheses for input: 'C[C@@H](CO)N1C[C@@H](C)[C@@H](CN(C)S(=O)(=O)c2ccc(C'
[11:09:16] SMILES Parse Error: extra open parentheses for input: 'C[C@H]1CN([C@H](C)CO)C(=O)c2ccccc2-c2ccccc2CO[C@@H]1CN(C)S(=O)(=O)c1ccc(C'
[11:09:16] SMILES Parse Error: extra open parentheses for input: 'C[C@H]1CN([C@@H](C)CO)C(=O)c2c(c3ccccc3n2C)-c2ccccc2CO[C@@H]1CN(C)S(=O)(=O)c1ccc(C'
[11:09:16] SMILES Parse Error: extra open parentheses for input: 'C[C@@H](CO)N1C[C@@H](C)[C@H](CN(C)S(=O)(=O)c2ccc(C'
[11:09:16] SMILES Parse Error: extra open parentheses for input: 'C[C@H]1CN([C@@H](C)CO)C(=O)c2ccccc2-c2ccccc2CO[C@H]1CN(C)S(=O)(=O)c1ccc(C'
[11:09:16] SMILES Parse Error: extra open parentheses for input: 'C[C@@H]1CN([C@H](C)CO)C(=O)c2c(c3ccccc3n2C)-c2ccccc2CO[C@@H]1CN(C)S(=O)(=O)c1ccc(C'
[11:09:16] SMILES Parse Error: extra open parentheses for input: 'C[C@@H]1CN([C@H](C)CO)C(=O)c2ccccc2-c2ccccc2CO[C@H]1CN(C)S(=O)(=O)c1ccc(C'
[11:09:16] SMILES Parse Error: extra ope

[11:09:17] SMILES Parse Error: extra open parentheses for input: 'CC1(C)[C@@H](/C=C(\Cl)C(F)(F)F)[C@H]1C(=O)O[C@H](C'
[11:09:17] SMILES Parse Error: extra open parentheses for input: 'Cn1c2ccccc2c2c3c(c4c5ccccc5n(CCC'
[11:09:17] SMILES Parse Error: extra open parentheses for input: 'Clc1cc(Cl)c(OCC'
[11:09:17] SMILES Parse Error: extra open parentheses for input: 'CC(C)[C@@H](C'
[11:09:17] WARNING: not removing hydrogen atom without neighbors
[11:09:17] SMILES Parse Error: extra open parentheses for input: 'CCOP(=O)(Cc1ccc(C(=O)Nc2ccc(Br)cc2C'
[11:09:17] SMILES Parse Error: extra open parentheses for input: 'Cc1ccc(OCC(=O)NNC(=O)CSc2ncc(C'
[11:09:17] SMILES Parse Error: extra open parentheses for input: 'COc1cccc(S(=O)(=O)NC2CCN(C'
[11:09:17] SMILES Parse Error: extra open parentheses for input: 'CCn1ccc(C(=O)NC2=C(C'
[11:09:17] SMILES Parse Error: extra open parentheses for input: 'Cc1ccc(C)c(S(=O)(=O)N2CCN(CC(=O)Nc3sccc3C'
[11:09:17] SMILES Parse Error: extra open parentheses for inp

[11:09:19] SMILES Parse Error: extra open parentheses for input: 'Cc1cccc(C(=O)NCN2CCN(c3ccccc3C'
[11:09:19] SMILES Parse Error: extra open parentheses for input: 'CCCCNC(=O)C(C'
[11:09:19] SMILES Parse Error: extra open parentheses for input: 'COc1ccccc1N1C(=C(C'
[11:09:19] SMILES Parse Error: extra open parentheses for input: 'COc1ccccc1N1CCS(=O)C1=C(C'
[11:09:19] SMILES Parse Error: extra open parentheses for input: 'C[N+](C)(CCCCC(C'
[11:09:19] SMILES Parse Error: extra open parentheses for input: 'C[C@H](CN(C(=O)c1ccc(C'
[11:09:19] SMILES Parse Error: extra open parentheses for input: 'CCc1nn(CCO)c(CC)c1Oc1cc(C'
[11:09:19] SMILES Parse Error: extra open parentheses for input: 'C[C@@H]1CC(=O)NN=C1c1ccc(NN=C(C'
[11:09:19] SMILES Parse Error: extra open parentheses for input: 'CCOc1cc2ncc(C'
[11:09:19] SMILES Parse Error: extra open parentheses for input: 'CCOC(=O)C(=O)Nc1cc(C'
[11:09:19] SMILES Parse Error: extra open parentheses for input: 'CCCCC([N+]'
[11:09:19] SMILES Parse Error

[11:09:35] SMILES Parse Error: extra open parentheses for input: 'CC1=CC=C(C'
[11:09:35] SMILES Parse Error: extra open parentheses for input: 'CC1=C2[N+]3=C(C=C4[N+]5=C(C(C)=C6N7[C@@H]8[C@H](CC(N)=O)[C@]6(C)CCC(=O)NC[C@H](C)OP(=O)([O-])O[C@@H]6[C@@H](CO)O[C@@H]([C@@H]6O)n6c[n+](c9ncnc(N)c96)[Co-3]753(C'
[11:09:35] SMILES Parse Error: extra open parentheses for input: 'CC1=C2[N+]3=C(C=C4[N+]5=C(C(C)=C6N7[C@@H]8[C@H](CC(N)=O)[C@]6(C)CCC(=O)NC[C@H](C)OP(=O)([O-])O[C@@H]6[C@@H](CO)O[C@@H]([C@@H]6O)n6c[n+](c9nc(C)nc(N)c96)[Co-3]753(C'
[11:09:35] SMILES Parse Error: extra open parentheses for input: 'COc1cc2c(cc1C)n1c[n+]2[Co-3]234(C'
[11:09:35] SMILES Parse Error: extra open parentheses for input: 'CC1=C2[N+]3=C(C=C4[N+]5=C(C(C)=C6N7[C@H]([C@H](CC(N)=O)[C@@]6(C)CCC(=O)NC[C@@H](C)OP(=O)([O-])O[C@@H]6[C@@H](CO)O[C@H](Oc8ccccc8)[C@@H]6O)[C@]6(C)[N+](=C1[C@@H](CCC(N)=O)[C@]6(C)CC(N)=O)[Co-2]753C'
[11:09:35] SMILES Parse Error: extra open parentheses for input: 'CC1=C2[N+]3=C(C=C4[N+]5=C(C(C)=C

In [11]:
deduped[deduped['std_smiles'].map(lambda x : '*' not in x)]

,ID,name,reference,formula,charge,mass,InChI,InChIKey,SMILES,std_smiles,mol
1,MNXM01,PMF,mnx:PMF,H,1.0,1.00794,InChI=1S/p+1,InChIKey=GPRLSGONYQIRFK-UHFFFAOYSA-N,[H+],[H+],<rdkit.Chem.rdchem.Mol object at 0x282e69540>
2,MNXM02,OH(-),mnx:HYDROXYDE,OH,-1.0,17.00734,InChI=1S/H2O/h1H2/p-1,InChIKey=XLYOFNOQVPJJNP-UHFFFAOYSA-M,[O-][H],O,<rdkit.Chem.rdchem.Mol object at 0x282e69700>
5,MNXM10,NADH,chebi:57945,C21H27N7O14P2,-2.0,663.11022,InChI=1S/C21H29N7O14P2/c22-17-12-19(25-7-24-17...,InChIKey=BOPGDPNILDQYTO-NNYOXOHSSA-L,NC(=O)C1=CN([C@@H]2O[C@H](COP(=O)([O-])OP(=O)(...,NC(=O)C1=CN([C@@H]2O[C@H](COP(=O)(O)OP(=O)(O)O...,<rdkit.Chem.rdchem.Mol object at 0x282e69620>
6,MNXM100,(2E)-geranyl diphosphate,chebi:58057,C10H17O7P2,-3.0,311.04660,InChI=1S/C10H20O7P2/c1-9(2)5-4-6-10(3)7-8-16-1...,InChIKey=GVVPGTZRZFNKDS-JXMROGBWSA-K,CC(C)=CCC/C(C)=C/COP(=O)([O-])OP(=O)([O-])[O-],CC(C)=CCC/C(C)=C/COP(=O)(O)OP(=O)(O)O,<rdkit.Chem.rdchem.Mol object at 0x282e69690>
7,MNXM100000,"6Z,9Z-Heneicosadien-11-one",chebi:165749,C21H38O,0.0,306.29227,InChI=1S/C21H38O/c1-3-5-7-9-11-13-15-17-19-21(...,InChIKey=ZNAIFUOOHZIIGO-OHNCOSGTSA-N,CCCCC/C=C\C/C=C\C(=O)CCCCCCCCCC,CCCCC/C=C\C/C=C\C(=O)CCCCCCCCCC,<rdkit.Chem.rdchem.Mol object at 0x282e69770>
...,...,...,...,...,...,...,...,...,...,...,...
1292148,MNXM999995,"1-(14Z,17Z,20Z,23Z,26Z-dotriacontapentaenoyl)-...",slm:000692383,C69H119NO9P,-1.0,1136.86279,InChI=1S/C69H120NO9P/c1-4-7-10-13-16-19-22-24-...,InChIKey=HPLDUMASIOWCCR-LAENLQBASA-M,CC/C=C\C/C=C\C/C=C\CCCCCCCC(=O)NCCOP(=O)([O-])...,CC/C=C\C/C=C\C/C=C\CCCCCCCC(=O)NCCOP(=O)(O)OC[...,<rdkit.Chem.rdchem.Mol object at 0x413da2b20>
1292149,MNXM999996,"1-(14Z,17Z,20Z,23Z,26Z-dotriacontapentaenoyl)-...",slm:000692384,C73H121NO9P,-1.0,1186.87844,InChI=1S/C73H122NO9P/c1-4-7-10-13-16-19-22-24-...,InChIKey=VNZHXXLXDVSBLA-IZNAGHOASA-M,CC/C=C\C/C=C\C/C=C\C/C=C\C/C=C\C/C=C\CCC(=O)NC...,CC/C=C\C/C=C\C/C=C\C/C=C\C/C=C\C/C=C\CCC(=O)NC...,<rdkit.Chem.rdchem.Mol object at 0x413da2b90>
1292150,MNXM999997,"1-(14Z,17Z,20Z,23Z,26Z-dotriacontapentaenoyl)-...",slm:000692385,C69H121NO9P,-1.0,1138.87844,InChI=1S/C69H122NO9P/c1-4-7-10-13-16-19-22-24-...,InChIKey=FTBDAPNXHPOOLH-RUXWUTLCSA-M,CCCCC/C=C\C/C=C\C/C=C\C/C=C\C/C=C\CCCCCCCCCCCC...,CCCCC/C=C\C/C=C\C/C=C\C/C=C\C/C=C\CCCCCCCCCCCC...,<rdkit.Chem.rdchem.Mol object at 0x413da2c00>
1292151,MNXM999998,"1-(14Z,17Z,20Z,23Z,26Z-dotriacontapentaenoyl)-...",slm:000692386,C69H119NO9P,-1.0,1136.86279,InChI=1S/C69H120NO9P/c1-4-7-10-13-16-19-22-24-...,InChIKey=UTTKGJJHRRHZRR-BNJOEXAFSA-M,CCCCC/C=C\C/C=C\C/C=C\C/C=C\C/C=C\CCCCCCCCCCCC...,CCCCC/C=C\C/C=C\C/C=C\C/C=C\C/C=C\CCCCCCCCCCCC...,<rdkit.Chem.rdchem.Mol object at 0x413da2c70>
